vector and count the words in tweets.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func, inspect
from sqlalchemy import Table, Column, Integer, String, Float, DateTime, MetaData

In [13]:
# Create an engine for the  database

engine = create_engine("sqlite:///../data/data.sqlite", echo=False)
conn = engine.connect()

# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
meta = MetaData()

tweets = Table(
   'tweets', meta, 
    Column('source',String), 
    Column('text', String), 
    Column('created_at', DateTime),
    Column('retweet_count', Integer),
    Column('favorite_count', Integer),
    Column('id_str', Integer, primary_key = True)
)

no_retweets = Table(
   'no_retweets', meta, 
    Column('source',String), 
    Column('text', String), 
    Column('created_at', DateTime),
    Column('retweet_count', Integer),
    Column('favorite_count', Integer),
    Column('id_str', Integer, primary_key = True)
)


In [14]:
#- select column 'text' from the tweets table - this has no retweets
tweets_text_data = pd.read_sql('SELECT TEXT FROM TWEETS', conn)


In [15]:
tweets = tweets_text_data.text

In [16]:
# Instantiate Count Vectorizer
#vectorizer = CountVectorizer( lowercase=True, token_pattern=r'\b[^\d\W]+\b', stop_words='english')
from sklearn.feature_extraction import text 

my_additional_stop_words = ['https','rt']

stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
vectorizer = CountVectorizer( lowercase=True, stop_words=stop_words)

vector_text = tweets

# To actually create the vectorizer, we simply need to call fit on the text
# data that we wish to fix
cv_fit = vectorizer.fit_transform(vector_text)

# Now, we can inspect how our vectorizer vectorized the text
# This will print out a list of words used, and their index in the vectors
# print('Vocabulary: ')
# print(vectorizer.vocabulary_)

# # If we would like to actually create a vector, we can do so by passing the
# # text into the vectorizer to get back counts
# vector = vectorizer.transform(vector_text)

# # Our final vector:
# print('Full vector: ')
# print(vector.toarray())

# # Or if we wanted to get the vector for one word:
# print('Hot vector: ')
# print(vectorizer.transform(['hot']).toarray())

# # Or if we wanted to get multiple vectors at once to build matrices
# print('Hot and one: ')
# print(vectorizer.transform(['hot', 'one']).toarray())

# # We could also do the whole thing at once with the fit_transform method:
# print('One swoop:')
# new_text = ['Today is the day that I do the thing today, today']
# new_vectorizer = CountVectorizer()
# print(new_vectorizer.fit_transform(new_text).toarray())

In [17]:
words = vectorizer.vocabulary_

In [18]:
feature_names = vectorizer.get_feature_names()
feature_counts = cv_fit.toarray().sum(axis=0)




In [20]:
final_count_df = pd.DataFrame({"word": feature_names, "cnt_word": feature_counts})
final_count_df.sort_values(by='cnt_word', ascending=False)

,word,cnt_word
21819,realdonaldtrump,4157
11616,great,4067
27046,trump,3229
2693,amp,2790
20791,president,2055
26219,thank,1913
20063,people,1854
14767,just,1707
2647,america,1446
6692,country,1346


In [22]:
final_count_df.to_csv('../data/final_word_count.csv', index=False)